In [ ]:
from pyspark.mllib.clustering import KMeans
from pyspark.mllib.clustering import GaussianMixture
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark import SparkContext
from numpy import array
from math import sqrt
from pyspark.ml.pipeline import Pipeline
import pandas as pd
import sys
import pyspark


file = "data/0.feat"

data = sc.textFile(file)
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

clusters = KMeans.train(parsedData, 6, maxIterations=20, runs=30,  initializationMode="k-means||")

vecAssembler = VectorAssembler(inputCols=[], outputCol="features")
scaler = StandardScaler(withMean=True, withStd=True, inputCol="features", outputCol="scaled_features")
firstPipeline = Pipeline(stages=[vecAssembler, clusters])

centroids = clusters.centers
weights = clusters.k
cost = clusters.computeCost(parsedData)

def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

gmm = GaussianMixture.train(parsedData, 2)
for i in range(2):
    print ("weight = ", gmm.weights[i], "mu = ", gmm.gaussians[i].mu,
        "sigma = ", gmm.gaussians[i].sigma.toArray())
    
    
#corpus = parsedDataForLDA.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()



%matplotlib inline
import ggplot as gg





In [ ]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import colorsys
import matplotlib.pylab as pylab
import glob

pylab.rcParams['figure.figsize'] = 12, 12  

def rgb_to_hex(rgb):
    return '#' + ''.join(['%02x' % int(p * 255) for p in rgb])

def get_colours(count):
    count = count + 1
    colours = [colorsys.hsv_to_rgb(h / count, 1, 1) for h in range(count)]
    return [rgb_to_hex(c) for c in colours]

def draw_graph(graph):
    nodes = set([n1 for n1, n2 in graph] + [n2 for n1, n2 in graph])
    G=nx.Graph()
    for node in nodes:
        G.add_node(node)
    for edge in graph:
        G.add_edge(edge[0], edge[1])
    pos = nx.spring_layout(G, k=0.25)
    
    c = list(nx.k_clique_communities(G, 10))
    rainbow = get_colours(len(c))
    colours = ['r' for n in G.nodes()]
    for node in G.nodes():
        colours[G.nodes().index(node)] = rainbow[len(c)]
        for community in c:
            if node in community:
                colours[G.nodes().index(node)] = rainbow[c.index(community)]
    
    nx.draw_networkx(G,
                     pos,
                     node_size=100,
                     linewidths=0.5,
                     with_labels=False,
                     node_color=colours) 
    
    plt.axis("off")
    plt.show()

def read_file(name):
    with open(name, "r") as f:
        return[list(map(int, line.split())) for line in f.readlines()]
    

graph = [(20, 21),(21, 22),(22, 23), (23, 24),(24, 25), (25, 20)]
draw_graph(graph)
#draw_graph(read_file("data/0.edges")) 